# Solving SAT problem with Grover's algorithm Kata Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [Solving SAT problem with Grover's algorithm kata](./SolveSATWithGrover.ipynb). Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

**What you should know for this workbook**

You should be familiar with the following concepts before tackling the Solve SAT with Grover kata (and this workbook):

1. [Basic linear algebra](./../tutorials/LinearAlgebra/LinearAlgebra.ipynb).
2. [Single-qubit](./../tutorials/SingleQubitGates/SingleQubitGates.ipynb) and [multi-qubit quantum gates](./../tutorials/MultiQubitGates/MultiQubitGates.ipynb) and using them to manipulate the state of the system.
3. [Exploring Grover's algorithm tutorial](./../tutorials/ExploringGroversAlgorithm/ExploringGroversAlgorithmTutorial.ipynb) and [Grover's algorithm implementation kata](./../GroversAlgorithm/GroversAlgorithm.ipynb).
4. Basic knowledge of logic gates and SAT problems.

## Part I. Oracles for SAT problems

### Task 1.1. The AND oracle: $f(x) = x_0 \wedge x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if all qubits of the query register are in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

The goal is to flip the qubit $|y\rangle$ if and only if all qubits in the register $|x\rangle$ are in the state $|1\rangle$.  
Hence the required unitary $U_{and}$ is such that:
$$U_{and}|x\rangle|y\rangle = \begin{cases} 
          |x\rangle|y\rangle & \text{if }x \neq 1...1 \\
          |x\rangle X|y\rangle & \text{if }x = 1...1 
       \end{cases}$$
       
We can rewrite this as 

$$U_{and} = \big(I - |1...1\rangle\langle 1...1|\big) \otimes I + |1...1\rangle\langle 1...1|\otimes X$$ 

Here we're using a convenient shorthand for ket-bra notation of gate effect on multiple basis states at once:

$$I - |1...1\rangle\langle 1...1| = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i|$$

For the number of qubits in the input register $n=2$, $U_{and}$ is the **Toffoli** gate:
$$U_{and} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
\end{bmatrix}$$

In the general case of arbitrary number of qubits in the input, 
this can be represented as a `Controlled X` gate, with the input register $|x\rangle$ as control and the target qubit $|y\rangle$ as target. 

> In Q# you can use [Controlled functor](https://docs.microsoft.com/azure/quantum/user-guide/language/expressions/functorapplication#controlled-functor) to apply a controlled version of the gate, if the gate specifies it.
>
> If $|x\rangle$ consists of just 2 qubits, we could also use the built-in **Toffoli** gate - the [`CCNOT` gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.ccnot).

In [ ]:
%kata T11_Oracle_And 

operation Oracle_And (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    Controlled X(queryRegister, target); 
}

[Return to task 1.1 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-1.1.-The-AND-oracle:-$f(x)-=-x_0-\wedge-x_1$)

### Task 1.2. The OR oracle: $f(x) = x_0 \vee x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if at least one qubit of the query register is in the $|1\rangle$ state, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

The goal is to flip the qubit $|y\rangle$ if at least one qubit in the register $|x\rangle$ is in the state $|1\rangle$.  
This can also be restated as to flip the qubit $|y\rangle$ *unless* the register $|x\rangle$ is in the state $|0...0\rangle$.

Hence the required unitary $U_{or}$ is such that:
$$U_{or}|x\rangle|y\rangle = \begin{cases} 
          |x\rangle |y\rangle & \text{if }x = 0...0  \\
          |x\rangle X|y\rangle & \text{if }x \neq 0...0
       \end{cases}$$

We can rewrite this as
$$U_{or} = |0...0\rangle\langle 0...0|\otimes I + \big(I - |0...0\rangle\langle 0...0|\big) \otimes X$$  

We can now reduce this problem to the AND oracle using the fact that OR gate can be created using an AND gate and NOT gates.  

$$x_0\vee x_1\vee ... \vee x_{n-1} = \lnot(\lnot x_0\wedge \lnot x_1\wedge ... \wedge \lnot x_{n-1})$$

Thus $U_{or}$ can be rewritten as $U_{or}=(X^{\otimes n}\otimes X) \cdot U_{and} \cdot (X^{\otimes n}\otimes I)$. 

> <details>
    <summary><b>Click here to see how to get this matrix expression using ket-bra notation</b></summary>
>    
$$
U_{or}= (X^{\otimes n}\otimes X) \cdot U_{and} \cdot (X^{\otimes n}\otimes I) = \\
 = (X^{\otimes n}\otimes X) \cdot (\sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i| \otimes I + |1...1\rangle\langle 1...1|\otimes X) \cdot (X^{\otimes n}\otimes I) = \\
 = (X^{\otimes n}\otimes X) \cdot (\sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i|X^{\otimes n} \otimes I + |1...1\rangle\langle 1...1|X^{\otimes n}\otimes X) = \\
 = (X^{\otimes n}\otimes X) \cdot (\sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}|i\rangle\langle i|X^{\otimes n} \otimes I + |1...1\rangle\langle 0...0|\otimes X) = \\
 = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}X^{\otimes n}|i\rangle\langle i|X^{\otimes n} \otimes X + X^{\otimes n}|1...1\rangle\langle 0...0|\otimes I = \\
 = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |1...1\rangle}X^{\otimes n}|i\rangle\langle i|X^{\otimes n} \otimes X + |0...0\rangle\langle 0...0|\otimes I = \\
 = \sum_{i \in \{0,1\}^n,\\ |i\rangle\neq |0...0\rangle}|i\rangle\langle i| \otimes X + |0...0\rangle\langle 0...0|\otimes I$$
</details>

This corresponds to the following steps:

1. Flip all the qubits of $|x\rangle$ using X gates to convert the inputs from $|x_0, x_1, ..., x_{n-1}\rangle$ to $|\lnot x_0, \lnot x_1, ..., \lnot x_{n-1}\rangle$.
2. Apply $U_{and}$ oracle to convert $|y\rangle$ to $|y \oplus (\lnot x_0\wedge \lnot x_1\wedge ... \wedge \lnot x_{n-1}) \rangle$.
3. Undo the flipping to return the inputs to $|x_0, x_1, ..., x_{n-1}\rangle$.
4. Flip the state of the target qubit $|y\rangle$ again to perform the final negation $|y \oplus \lnot(\lnot x_0\wedge \lnot x_1\wedge ... \wedge \lnot x_{n-1}) \rangle = |y \oplus x_0\vee x_1\vee ... \vee x_{n-1} \rangle$.

> We can use the ` within ... apply ...` construct in Q# implement the pattern of modifying the qubits of $|x\rangle$, applying the oracle to get the result and write it into $|y\rangle$ and undoing (uncomputing) the modification to $|x\rangle$.

In [ ]:
%kata T12_Oracle_Or 

operation Oracle_Or (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    within {
        // Flip input qubits to negate them
        ApplyToEachA(X, queryRegister);
    } apply {
        // Use the AND oracle to get the AND of negated inputs
        Oracle_And(queryRegister, target);
    }
    // Flip the target to get the final answer
    X(target);
}

[Return to task 1.2 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-1.2.-The-OR-oracle:-$f(x)-=-x_0-\vee-x_1$)

### Task 1.3. The XOR oracle: $f(x) = x_0 \oplus x_1$

**Inputs:** 

  1. 2 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
i.e., flip the target state if the qubits of the query register are in different states, 
and leave it unchanged otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

Let's consider the case of $n$-qubit register $|x\rangle$; then $f(x) = x_0 \oplus x_1 \oplus ... \oplus x_{n-1}$. 
The goal is to flip the qubit $|y\rangle$ if and only if $f(x) = 1$.  

Hence the effect of the required unitary $U_{xor}$ on the basis state $|x\rangle|y\rangle$ will be:

$$U_{xor}|x\rangle|y\rangle = |x\rangle|y\oplus f(x)\rangle = \\
= |x\rangle|y\oplus x_0 \oplus x_1 \oplus ... \oplus x_{n-1}\rangle = \\
= |x\rangle|(...((y\oplus x_0) \oplus x_1) ... )\oplus x_{n-1}\rangle$$

Let's consider a unitary $U_{\oplus i}$, which has the following effect on the basis state:

$$U_{\oplus i}|x\rangle|y\rangle = |x\rangle|y \oplus x_i\rangle$$

This unitary is simply a CNOT gate with i-th qubit $|x_i\rangle$ as control and $|y\rangle$ as target.

Then we can rewrite the effect of our unitary $U_{xor}$ as follows:

$$U_{xor}|x\rangle|y\rangle = U_{\oplus n-1} (U_{\oplus n-2} (... U_{\oplus 0} |x\rangle|y\rangle ...)) = \prod_{i=0}^{n-1}U_{\oplus i}|x\rangle|y\rangle$$

We see that we can write

$$U_{xor}=\prod_{i=0}^{n-1}U_{\oplus i}$$

We can implement this as a sequence of CNOT gates, with each of the qubits of the input register as controls and the output qubit as target.

In [ ]:
%kata T13_Oracle_Xor 

operation Oracle_Xor (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    ApplyToEachA(CNOT(_, target), queryRegister); 
}

[Return to task 1.3 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-1.3.-The-XOR-oracle:-$f(x)-=-x_0-\oplus-x_1$)

### Task 1.4. Alternating bits oracle: $f(x) = (x_0 \oplus x_1) \wedge (x_1 \oplus x_2) \wedge \dots \wedge (x_{N-2} \oplus x_{N-1})$

**Inputs:** 

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

### Solution

Let $x_i'=x_i\oplus x_{i+1}$, then we can rewrite $f(x)$ as $f(x) = x_0' \wedge x_1' \wedge ... \wedge x_{N-2}'$.

We can compute $x_i'$ using $U_{xor}$ (implemented by operation `Oracle_Xor`) from the previous task, since $U_{xor}|x_l\rangle|x_r\rangle|0\rangle = |x_l\rangle|x_r\rangle|x_l \oplus x_r\rangle$.
We will store XORs of pairs of adjacent qubits in an auxiliary register of qubits that we'll allocate for this purpose. (This procedure can be undone in the same way before deallocating that register.)

Now we can find $f(x)$ using the values $x_i'$, i.e., transform $|y\rangle$ into $|y \oplus f(x)\rangle$ using the `Oracle_And` on the auxiliary register and the `target` qubit.

Hence the final solution consists of:
1. Finding XORs of all pairs of adjacent qubits and storing them in an auxiliary register.
2. Applying $U_{and}$ (`Oracle_And`) on `auxiliaryRegister` and `target`.
3. Undo the computation done in 1. to reset the `auxiliaryRegister` before releasing the qubits.

We can use the `within ... apply ...` construct in Q# to express this sequence of steps, similar to task 2.

In [ ]:
%kata T14_Oracle_AlternatingBits 

open Microsoft.Quantum.Arrays;

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(queryRegister);
    use auxiliaryRegister = Qubit[N-1];
    // Find the XOR of all pairs of consecutive qubits and store the result in auxiliaryRegister
    within {
        for i in 0 .. N-2 {
            Oracle_Xor(queryRegister[i .. i+1], auxiliaryRegister[i]);
        }
    }
    // Use the And oracle on auxiliaryRegister to get the answer
    apply {
        Oracle_And(auxiliaryRegister, target); 
    }
}

We can also avoid using the auxiliary register by computing XORs in place, i.e., storing the results of XOR operations in the `queryRegister` itself. 
To do this, we apply $\textrm{CNOT}$ to two adjacent qubits, with the target qubit being the one in which we store the result. 

Since all qubits except those at index $0$ and $N-1$ are inputs to two XOR operations, the order of these operations must be carefully chosen.
If we are going from index $0$ to $N-1$ when computing $x_i'$, the target must always be the lower index (the one that will not be involved in later computations), i.e., `CNOT(queryRegister[i+1], queryRegister[i])`. This way `queryRegister[i+1]` is preserved for the next XOR operation.

Now the `queryRegister[0..N-2]` contains the XOR results, and we can perform the `Oracle_And` operation with this array as control directly. We uncompute the effects of the XORs computations using `within ... apply ...` construct, as usual.

In [ ]:
%kata T14_Oracle_AlternatingBits 

open Microsoft.Quantum.Arrays;

operation Oracle_AlternatingBits (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    let N = Length(queryRegister);
    // Find the XOR of all pairs of consecutive qubits and store the result in queryRegister itself
    within {
        for i in 0 .. N-2 {
            CNOT(queryRegister[i+1], queryRegister[i]);
        }
    }
    // Use the And oracle on the first N-1 qubits of queryRegister to get the answer
    apply {
        Oracle_And(Most(queryRegister),target); 
    }
}

[Return to task 1.4 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-1.4.-Alternating-bits-oracle:-$f(x)-=-(x_0-\oplus-x_1)-\wedge-(x_1-\oplus-x_2)-\wedge-\dots-\wedge-(x_{N-2}-\oplus-x_{N-1})$)

### Task 1.5. Evaluate one clause of a SAT formula

> For SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of several clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$clause(x) = \bigvee_k y_{k},\text{ where }y_{k} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$
>
> For example, one of the possible clauses on two variables is:
>
> $$clause(x) = x_0 \vee \neg x_1$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 1-dimensional array of tuples `clause` which describes one clause of a SAT problem instance $clause(x)$.

`clause` is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

* The clause $x_0 \vee \neg x_1$ can be represented as `[(0, true), (1, false)]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

### Solution

This task involves evaluating a clause which is a disjunction (OR) of negated and non-negated variables encoded in the `queryRegister` . This task can be solved in 2 steps using a similar technique to the one we saw in task 1.4:

1. We first flip the qubits which are negated in `clause` (and later undo this operation).  
   To do this, we conditionally apply an $X$ gate to qubit $|x_j\rangle$ if and only if the clause has a term of the form `(j, false)`.
2. We use the $U_{or}$ unitary (implemented by `Oracle_Or`) to calculate $|y\oplus f(x)\rangle$.  
   Here we need to construct an array `clause_qubits` - all qubits which are included as a negated or non-negated variable in the clause. 
We then apply the `Oracle_Or` operation on `clause_qubits` and `target`.

> We want our implementation of `Oracle_SATClause` be adjointable, and to rely on Q# compiler to generate the adjoint variant of this operation. To enable this, we move the classical computations that involve manipulating mutable variables to a separate function.

In [ ]:
// Find all qubits which are variables in clause
function GetClauseQubits (queryRegister : Qubit[], clause: (Int,Bool)[]) : Qubit[] {
    mutable clauseQubits = [];
    for (index, _) in clause {
        set clauseQubits += [queryRegister[index]];
    }
    return clauseQubits;
}

In [ ]:
%kata T15_Oracle_SATClause 

open Microsoft.Quantum.Arrays;
    
operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    within {
        // Flip all qubits which are in negated form in the SAT clause.
        for (index, positive) in clause {
            if (not positive) {
                X(queryRegister[index]);
            }
        }
    }
    apply {
        // Find all qubits which are variables in the clause.
        let clauseQubits = GetClauseQubits(queryRegister, clause); 
        // Compute disjunction of variables in clause.
        Oracle_Or(clauseQubits, target);
    }
}

We can also use several Q# library functions and constructs to write our code in the functional style.

* The functions 
[`Fst`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.fst) and
[`Snd`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.snd) 
extract the first and second element of a 2-element tuple, respectively. 
`Fst(term)` would give us the index $j$ for variable $x_j$ and `Snd(term)` would be `false` if the variable was negated or `true` if it was not. 
* We can use the function
[`Mapped`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arrays.mapped) 
to extract the list of variable indices used in the clause. Given an array and a function that is defined for the elements of the array, `Mapped` returns a new array that consists of the images of the original array under the function. `Mapped(Fst<Int, Bool>, clause)` will return an array of all first elements of the tuples in `clause`.
* Finally, we use the function 
[`Subarray`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arrays.subarray) to find the `clauseQubits`: this function takes an array and a list of indices and produces a new array formed from the elements of the original array at the given locations.

In [ ]:
%kata T15_Oracle_SATClause 

open Microsoft.Quantum.Arrays;
    
operation Oracle_SATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    // Flip all qubits which are in negated form in the SAT clause.
    within {
        for term in clause {
            // If Snd(term) is false, apply X gate, otherwise apply I (do nothing).
            (Snd(term)? I | X)(queryRegister[Fst(term)]);
        }
    }
    apply {
        // Get indices of all variables in clause.
        let indexList = Mapped(Fst<Int, Bool>, clause);
        // Get all qubits which are part of clause.
        let clauseQubits = Subarray(indexList,queryRegister);
        // Compute disjunction of variables in clause.
        Oracle_Or(clauseQubits,target);
    }
}

[Return to task 1.5 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-1.5.-Evaluate-one-clause-of-a-SAT-formula)

### Task 1.6. k-SAT problem oracle

> For k-SAT problems, $f(x)$ is represented as a conjunction (an AND operation) of $M$ clauses on $N$ variables,
and each clause is a disjunction (an OR operation) of **one or several** variables or negated variables:
>
> $$f(x) = \bigwedge_i \big(\bigvee_k y_{ik} \big),\text{ where }y_{ik} =\text{ either }x_j\text{ or }\neg x_j\text{ for some }j \in \{0, \dots, N-1\}$$

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the k-SAT problem instance $f(x)$.

$i$-th element of `problem` describes the $i$-th clause of $f(x)$; 
it is an array of one or more tuples, each of them describing one component of the clause.

Each tuple is an `(Int, Bool)` pair:

* the first element is the index $j$ of the variable $x_j$, 
* the second element is `true` if the variable is included as itself ($x_j$) and `false` if it is included as a negation ($\neg x_j$).

**Example:**

A more general case of the OR oracle for 3 variables $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

### Solution

This task consists of a conjunction (AND) of results of multiple clause evaluations. Each clause individually can be evaluated using the code we've written in task 1.5. The computation results of these clauses must be stored (temporarily) in freshly allocated qubits. Then the conjunction of these results can be computed using `Oracle_And` from task 1.1.

Let the number of clauses in the formula be $m$, then the steps for implementing k-SAT oracle will be:
1. Allocate an array of $m$ qubits in the state $|0\rangle$ `auxiliaryRegister`.
2. Evaluate each clause using `Oracle_SATClause` from task 1.5 with the corresponding element of `auxiliaryRegister` as the target qubit.
3. Evaluate the SAT formula using `Oracle_And` implemented in task 1.1 with `auxiliaryRegister` as the input register and `target` as the target qubit.
4. Undo step 2 to restore the auxiliary qubits back into the $|0^{\otimes m}\rangle$ state before releasing them.

We can again use the `within ... apply ...` Q# language construct to perform steps 2, 3 and 4.

In [ ]:
%kata T16_Oracle_SAT 

operation Oracle_SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    use auxiliaryRegister = Qubit[Length(problem)];
    // Compute the clauses.
    within {
        for i in 0 .. Length(problem) - 1 {
            Oracle_SATClause(queryRegister, auxiliaryRegister[i], problem[i]);
        }
    }
    // Evaluate the overall formula using an AND oracle.
    apply {
        Oracle_And(auxiliaryRegister, target);
    }
}

[Return to task 1.6 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-1.6.-k-SAT-problem-oracle)

## Part II. Oracles for exactly-1 3-SAT problem

Exactly-1 3-SAT problem (also known as "one-in-three 3-SAT") is a variant of a general 3-SAT problem.
It has a structure similar to a 3-SAT problem, but each clause must have *exactly one* true literal, 
while in a normal 3-SAT problem each clause must have *at least one* true literal.

### Task 2.1. "Exactly one $|1\rangle$" oracle

**Inputs:** 

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:** 

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if exactly one bit of $x$ is in the $|1\rangle$ state, and $0$ otherwise.

Leave the query register in the same state it started in.

**Stretch Goal:** 

Can you implement the oracle so that it would work 
for `queryRegister` containing an arbitrary number of qubits?

### Solution

Consider the set of all bit strings of length $n$ which have only one bit of $x$ equal to $1$.   
This set of bit strings is $S=\{00...01, 00...10, ..., 00..1..00, ..., 01...00, 10...00\}$, 
or, if we convert the bit strings to integers, 
$$S=\{1,2,4,..2^i,..,2^{n-1}\} = \{2^k: 0 \le k \le n-1\}$$

We need to implement an oracle that flips $|y\rangle$ if the input basis state $|x\rangle$ corresponds to one of the bit strings in $S$. We can represent this as the following unitary in Dirac notation:

$$U = \sum_{i\in S}|i\rangle\langle i|\otimes X + \sum_{j \not\in S}|j\rangle\langle j|\otimes I$$

This unitary can be implemented using $n$ controlled NOT gates, with `queryRegister` as control and the `target` qubit flipped if and only if the control register is in a particular state of the form $2^k$.

We can use the Q# library function [`ControlledOnInt`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonint) to implement these controlled NOT gates. 

In [ ]:
%kata T21_Oracle_Exactly1One 

operation Oracle_Exactly1One (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    for i in 0 .. Length(queryRegister) - 1 {
        (ControlledOnInt(2^i, X))(queryRegister, target);
    }
}

[Return to task 2.1 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-2.1.-"Exactly-one-$|1\rangle$"-oracle)

### Task 2.2. "Exactly-1 3-SAT" oracle

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register).

  2. A qubit in an arbitrary state $|y\rangle$ (target qubit).

  3. A 2-dimensional array of tuples `problem` which describes the 3-SAT problem instance $f(x)$.

`problem` describes the problem instance in the same format as in task 1.6;
each clause of the formula is guaranteed to have exactly 3 terms.

**Goal:**

Transform state $|x,y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

Leave the query register in the same state it started in.

**Example:**

An instance of the problem $f(x) = (x_0 \vee x_1 \vee x_2)$ can be represented as `[[(0, true), (1, true), (2, true)]]`,
and its solutions will be `(true, false, false)`, `(false, true, false)` and `(false, false, true)`, 
but none of the variable assignments in which more than one variable is true, which are solutions for the general SAT problem.

### Solution

This task is similar to [task 1.6](#Task-1.6.-k-SAT-problem-oracle): it consists of a conjunction (AND) of results of multiple clause evaluations, but this time each clause has to be evaluated using the *Exactly One 1* policy we've implemented in task 2.1, rather than an `Oracle_Or` we used in task 1.5. We can reuse the function `GetClauseQubits` from task 1.5:

In [ ]:
open Microsoft.Quantum.Arrays;
    
operation Oracle_Exactly1OneSATClause (queryRegister : Qubit[], target : Qubit, clause : (Int, Bool)[]) : Unit is Adj {
    within {
        for (index, positive) in clause {
            if (not positive) {
                X(queryRegister[index]);
            }
        }
    }
    apply {
        // Find all qubits which are variables in the clause.
        let clauseQubits = GetClauseQubits(queryRegister, clause); 
        // Check if "Exactly one 1" criteria is satisfied.
        Oracle_Exactly1One(clauseQubits,target);
    }
}

The rest of the code is similar to task 1.6. See the [solution for task 1.6](#Task-1.6.-k-SAT-problem-oracle) for the discussion of implementation details.

In [ ]:
%kata T22_Oracle_Exactly1SAT 

operation Oracle_Exactly1_3SAT (queryRegister : Qubit[], target : Qubit, problem : (Int, Bool)[][]) : Unit is Adj {
    use auxiliaryRegister = Qubit[Length(problem)];
    // Compute the clauses.
    within {
        for i in 0 .. Length(problem) - 1 {
            Oracle_Exactly1OneSATClause(queryRegister, auxiliaryRegister[i], problem[i]);
        }
    }
    // Evaluate the overall formula using an AND oracle.
    apply {
        Oracle_And(auxiliaryRegister, target);
    }
}

[Return to task 2.2 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-2.2.-"Exactly-1-3-SAT"-oracle)

## Part III. Using Grover's algorithm for problems with multiple solutions

### Task 3.1. Using Grover's algorithm

**Goal:**

Implement Grover's algorithm and use it to find solutions to SAT instances from parts I and II. 

### Solution

This task can be solved  following steps (see the [GroversAlgorithm kata](../GroversAlgorithm/GroversAlgorithm.ipynb) for details):

1. Grover's algorithm requires a phase oracle, rather than the marking oracles we implemented in parts I and II, so we need to write Q# code that converts a marking oracle to a phase oracle.

In [ ]:
operation Oracle_Converter (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    use target = Qubit();
    // Put the target into the |-⟩ state and later revert the state
    within { 
        X(target);
        H(target); 
    }
    // Apply the marking oracle; since the target is in the |-⟩ state,
    // flipping the target if the register satisfies the oracle condition will apply a -1 factor to the state
    apply { 
        markingOracle(register, target);
    }
}

2. The main part of Grover's algorithm is a loop which runs the Grover's algorithm iteration with the given `markingOracle` on the given `register` the given number of times.
This loop results in a state on the given `register` which satisfies the condition marked by the marking oracle with high probability.

In [ ]:
open Microsoft.Quantum.Arrays;

operation GroversLoop (register: Qubit[], oracle: ((Qubit[], Qubit) => Unit is Adj), numIterations: Int) : Unit {
    let phaseOracle = Oracle_Converter(oracle, _);
    ApplyToEach(H, register);

    for _ in 1 .. numIterations {
        phaseOracle(register);
        within{
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        } 
        apply{
            Controlled Z(Most(register), Tail(register));
        }
    }
}

3. Finally, the operation `Run_GroversSearch_Algorithm` will define the problem you want to solve, run the `GroversLoop` and measure the qubit register to find the solution. 

The parameters defining size of register `N` and `oracle` are up to you!

Try experimenting with the number of iterations in the code. 
Remember that for an oracle which has $M << N, N=2^n$ marked states, the optimal number of iterations is $\frac{\pi}{4}\sqrt{\frac{N}{M}}$.

The solution given here runs Grover's search algorithm once, and if it fails to find the solution, you need to retry manually. You can add the logic to automate the retries; see [Exploring Grover's Algorithm tutorial](./../tutorials/ExploringGroversAlgorithm/ExploringGroversAlgorithm.ipynb) for details.

In [ ]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation Run_GroversSearch_Algorithm () : Unit {
    // Create a data structure describing the SAT instance we want to solve
    let problem = [[(0, true)], [(0, false), (1, false)]];
    let N = 2;

    // Define the oracle we want to use; try this with different oracles!
    let oracle = Oracle_SAT(_, _, problem);
    
    // Try different numbers of iterations.
    let numIterations = 1;
    use (register, output) = (Qubit[N], Qubit());
    Message($"Trying search with {numIterations} iterations");
    GroversLoop(register, oracle, numIterations);

    // Measure the result.
    let res = MultiM(register);

    // Apply the oracle to the register after measurement to check whether the result is correct. 
    oracle(register, output);

    if (MResetZ(output) == One) {
        let answer = ResultArrayAsBoolArray(res);
        Message($"Found Answer: {answer}");
    } else {
        Message("Failed to find an answer. Try again!");
    }
    ResetAll(register);        
}

In [ ]:
%simulate Run_GroversSearch_Algorithm

[Return to task 3.1 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-3.1.-Using-Grover's-algorithm)

### Task 3.2. Universal implementation of Grover's algorithm

**Inputs:**

  1. The number of qubits N.

  2. A marking oracle which implements a boolean expression, similar to the oracles from part I.

**Output:**

An array of N boolean values which satisfy the expression implemented by the oracle 
(i.e., any basis state marked by the oracle).

### Solution

This is very similar to task 3.1, but we don't know the problem described by the `oracle`, so we have no way of predicting the optimal number of iterations. Instead we try the series $\{1, 2, 4, 8, ...\}$ as the number of iterations. Even if none of these numbers are optimal, we will still eventually get the correct answer with high probability. (There are also other ways to pick the sequence of the numbers of iterations to try; feel free to experiment with them!)

We use the repeat-until-success (RUS) loop to try the search with different numbers of iterations until we find an answer. The `repeat` block (the body of the loop) consists of performing the `GroversLoop`, measuring the result and checking if it is correct. The `until` block defines the condition of exiting the loop - if we found a correct answer or reached a time-out condition. The `fixup` block happens before retrying the algorithm, in this case it doubles the number of iterations in order to try again.

In [ ]:
%kata T32_UniversalGroversAlgorithm 

open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation UniversalGroversAlgorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    // Since we are unaware of the optimal number of iterations, we try different numbers of iterations.
    mutable answer = [false, size = N];
    use (register, output) = (Qubit[N], Qubit());
    mutable correct = false;
    mutable iter = 1;
    repeat {
        Message($"Trying search with {iter} iterations");
        GroversLoop(register, oracle, iter);
        let res = MultiM(register);

        oracle(register, output);
        if (MResetZ(output) == One) {
            set correct = true;
            set answer = ResultArrayAsBoolArray(res);
        }
        ResetAll(register);
    } until (correct or iter > 10)  // The fail-safe to avoid going into an infinite loop
    fixup {
        set iter *= 2;
    }
    if (not correct) {
        fail "Failed to find an answer";
    }
    Message($"{answer}");
    return answer;
}


[Return to task 3.2 of the Solving SAT with Grover's algorithm kata.](./SolveSATWithGrover.ipynb#Task-3.2.-Universal-implementation-of-Grover's-algorithm)